In [ ]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

In [16]:
import numpy as np
# ASSUMPTION: Modelling date as an integer for simplicity
source = {"loc": ["Sangli","Nashik","Dhule"],
          "qty": [40,50,60],
          "date": [10,15,26]}
destination = {"loc": ["Powai","Mahim","Chembur"],
               "qty": [10,50,60],
               "date": [20,22,30]}
s_len = len(source)
d_len = len(destination)

# alpha: Indicator variables(0 or 1) whether source i is supplying dest j
alpha = np.arange(1,s_len*d_len+1)
alpha_idx = np.reshape(np.arange(1,s_len*d_len+1),(s_len,d_len))

# x: Quantity variable delivered by source i to dest j
x = np.arange(1,s_len*d_len+1)
x_idx = np.reshape(np.arange(1,s_len*d_len+1),(s_len,d_len))

# Very large number
M = 999999

In [29]:
# Building the model

from pyomo.environ import *

m = ConcreteModel()

m.alpha = Var(alpha,domain=Binary) # Variables
m.x = Var(x,domain=NonNegativeReals)

weight = 1
# The weight can be increased to give more priority to minimizing number of assignments
# Given instructions: First maximize demand met, minimize number of assignments secondarily
# Order of x(demand met) >>> Order of indicator variables alpha
# Therefore, value of weight=1 is deemed good for our requirements
m.obj = Objective(expr=sum(m.x[i+1] for i in range(len(x))) - 
                  weight*sum(m.alpha[i+1] for i in range(len(alpha))),sense=maximize) # Objective

m.constraints = ConstraintList()
for i in range(s_len):
  for j in range(d_len):
    m.constraints.add(expr=source["date"][i] <= destination["date"][j] + M*(1-m.alpha[alpha_idx[i,j]]))

for i in range(s_len):
  for j in range(d_len):
    m.constraints.add(expr=m.x[x_idx[i,j]] <= M*m.alpha[alpha_idx[i,j]])

for i in range(s_len):
  m.constraints.add(expr=sum(m.x[x_idx[i,j]] for j in range(d_len)) <= source["qty"][i])

for j in range(d_len):
  m.constraints.add(expr=sum(m.x[x_idx[i,j]] for i in range(s_len)) <= destination["qty"][j])

# m.pprint()

In [31]:
opt = SolverFactory('cbc')
result = opt.solve(m)
print("Status :",result.solver.status)
print("Termination condition :",result.solver.termination_condition)
x_optimal = np.reshape(np.arange(1,s_len*d_len+1),(s_len,d_len))
for i in range(s_len):
  for j in range(d_len):
    x_optimal[i,j] = m.x[x_idx[i,j]].value
print(x_optimal)

Status : ok
Termination condition : optimal
[[10  0  0]
 [ 0 50  0]
 [ 0  0 60]]
